# Library Import

In [ ]:
# Remove the hash below to install optuna
# python3 -m pip install optuna

# load external modules
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy import random


from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.metrics import mean_squared_error
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Data Preparation

In [ ]:
data_list = [[np.random.rand(), np.random.rand(), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, np.random.rand()-0.5, np.random.rand()-0.5]] # first row
for i in range(1, 10000):
    new_row = [0.8 * data_list[i-1][0] + data_list[i-1][1] + data_list[i-1][12],
               -0.8 * data_list[i-1][1] + data_list[i-1][13],
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
               data_list[i-1][12] + np.random.rand() - 0.5,
               data_list[i-1][13] + np.random.rand() - 0.5]
    data_list.append(new_row)
d_o = pd.DataFrame(data_list) # to dataframe

for i in range(0, len(d_o)):
    d_o.loc[i, 2] = d_o.loc[i, 0] + 0.5 * d_o.loc[i, 1]
    d_o.loc[i, 3] = 0.3 * d_o.loc[i, 0] + d_o.loc[i, 1] + 0.4 * d_o.loc[i, 2] * d_o.loc[i, 2]
    d_o.loc[i, 4] = 0.5 * d_o.loc[i, 0] * d_o.loc[i, 1] - d_o.loc[i, 2] - 0.2 * d_o.loc[i, 3] * d_o.loc[i, 3]
    d_o.loc[i, 5] = d_o.loc[i, 1] * d_o.loc[i, 4] - 0.6 * math.tanh(d_o.loc[i, 0] * d_o.loc[i, 3]) + d_o.loc[i, 2]
    d_o.loc[i, 6] = 0.2 * d_o.loc[i, 0] * d_o.loc[i, 5] + math.tanh(d_o.loc[i, 1] + d_o.loc[i, 2]) + 0.4 * math.tanh(d_o.loc[i, 3] * d_o.loc[i, 4])
    d_o.loc[i, 7] = 0.35 * d_o.loc[i, 6] * d_o.loc[i, 4] - 0.4 * d_o.loc[i, 0] * d_o.loc[i, 5] + math.tanh(d_o.loc[i, 1] + d_o.loc[i, 2]) - 0.8 * d_o.loc[i, 1] * d_o.loc[i, 1] + 0.5 * d_o.loc[i, 3]
    d_o.loc[i, 8] = d_o.loc[i, 0] - 0.1 * d_o.loc[i, 1] * d_o.loc[i, 1] + 0.3 * d_o.loc[i, 2] * d_o.loc[i, 2] + d_o.loc[i, 3] * math.tanh(d_o.loc[i, 4] - d_o.loc[i, 5]) + math.tanh(d_o.loc[i, 6] * d_o.loc[i, 7])
    d_o.loc[i, 9] = d_o.loc[i, 0]+d_o.loc[i, 3] * d_o.loc[i, 6] + d_o.loc[i, 5] * math.tanh(d_o.loc[i, 4] - d_o.loc[i, 7]) - d_o.loc[i, 2] * math.tanh(d_o.loc[i, 1] + d_o.loc[i, 8])
    d_o.loc[i, 10] = 0.3 * d_o.loc[i, 5] + math.tanh(d_o.loc[i, 0] + d_o.loc[i, 8] * d_o.loc[i, 8]) - d_o.loc[i, 6] * math.tanh( d_o.loc[i, 1] -  d_o.loc[i, 9] *  d_o.loc[i, 9]) + 0.4 * ( d_o.loc[i, 2] *  d_o.loc[i, 2] +  d_o.loc[i, 3] +  d_o.loc[i, 4]) +  d_o.loc[i, 7]
    d_o.loc[i, 11] =0.45 * d_o.loc[i, 0] * d_o.loc[i, 2] - 0.6 * d_o.loc[i, 4] * d_o.loc[i, 1] + 0.1 * d_o.loc[i, 3] * d_o.loc[i, 3] * d_o.loc[i, 5] + math.tanh(d_o.loc[i, 6] / d_o.loc[i, 8] + d_o.loc[i, 7] * d_o.loc[i, 9]) + 0.5 * d_o.loc[i, 10]

scaler = StandardScaler()
data_to_scale = d_o.iloc[:, 1:]
data_standardized = scaler.fit_transform(data_to_scale)
columns_to_keep = d_o.columns[0]
data = pd.DataFrame(data_standardized, columns=data_to_scale.columns)
data.insert(loc=0, column=columns_to_keep, value=d_o[columns_to_keep])

x = data.iloc[:, 2:12]
u = data.iloc[:, 12:14]

In [ ]:
rc('font', family='Times New Roman')
# Plot columns in u and label them as u1 and u2
plt.figure(figsize=(8, 4), dpi=300)
for i in range(2):  # Assuming you want to plot columns 12 and 13
    plt.plot(u.iloc[:, i], label=f'u{i + 1}')
plt.title('u Columns')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.savefig('u_output.jpg', format='jpeg', dpi=300)
plt.show()

# Plot columns in x and label them as P3 to P12
plt.figure(figsize=(8, 4), dpi=300)
for i in range(10):  # Assuming you want to plot columns 2 to 11
    plt.plot(x.iloc[:, i], label=f'P{i + 3}')
plt.title('P Column')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.savefig('P_output.jpg', format='jpeg', dpi=300)
plt.show()

In [ ]:
x_train = x.iloc[:7999]
u_train = u.iloc[:7999]
P5_train = x.iloc[1:8000, 2]
P5_train.reset_index(drop=True, inplace=True)

x_test = x.iloc[7999:9999]
x_test.reset_index(drop=True, inplace=True)
u_test = u.iloc[7999:9999]
u_test.reset_index(drop=True, inplace=True)
P5_test = x.iloc[8000:, 2]
P5_test.reset_index(drop=True, inplace=True)

# VAE Model Build

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def encoder(x, units, activation, dropout):

    # forward pass the time series through the encoder network
    for i in range(len(units)):
        h = tf.keras.layers.Dense(units=units[i], activation=activation)(x if i == 0 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # derive the mean of the latent vector
    mu = tf.keras.layers.Dense(units=2)(h)

    # derive the standard deviation of the latent vector
    sigma = tf.exp(0.5 * tf.keras.layers.Dense(units=2)(h))

    # generate the latent vector
    z = mu + sigma * tf.random.normal(shape=(tf.shape(h)[0], 2))

    return mu, sigma, z


def decoder(z, features, units, activation, dropout):

    # forward pass the latent vector through the decoder network (note that the decoder has
    # the same layers as the encoder, but in reversed order)
    for i in range(1, 1 + len(units)):
        h = tf.keras.layers.Dense(units=units[-i], activation=activation)(z if i == 1 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # forward pass the output of the decoder network through a dense layer with number of
    # units equal to the number of time series; this ensures that the reconstructions have
    # the same shape as the inputs
    xhat = tf.keras.layers.Dense(units=features)(h)

    return xhat


def discriminator(z, units, activation, dropout):

    # forward pass the latent vector through the discriminator network
    for i in range(len(units)):
        h = tf.keras.layers.Dense(units=units[i], activation=activation)(z if i == 0 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # forward pass the output of the discriminator network through a dense layer with
    # one unit and with sigmoid activation in order to get a vector of probabilities
    p = tf.keras.layers.Dense(units=1, activation='sigmoid')(h)

    return p


class VAE():
    def __init__(self, data, units, activation, dropout):

        # difference the time series
        x = data.diff().iloc[1:, :]


        # scale the time series
        scaler = StandardScaler()
        scaler.fit(x)
        x = scaler.transform(x)

        # extract the number of samples
        samples = x.shape[0]

        # extract the number of time series
        features = x.shape[1]

        # build the model
        def build_model():

            tf.random.set_seed(42)

            # define the input
            x = tf.keras.layers.Input(shape=(features,))

            # forward pass the input through the encoder and get the latent representation
            mu, sigma, z = encoder(x, units, activation, dropout)



            # forward pass the latent representation through the decoder and get the reconstructions
            xhat = decoder(z, features, units, activation, dropout)

            # create the model
            model = tf.keras.models.Model(x, [mu, sigma, z, xhat])
            model.build(input_shape=(features,))

            return model

        # save the inputs
        self.x = x
        self.model = build_model()
        self.scaler = scaler
        self.samples = samples
        self.features = features


    def fit(self, batch_size, learning_rate, epochs, verbose):

        tf.random.set_seed(42)

        # instantiate the optimizer
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        # define the training loop
        @tf.function
        def train_step(x):
            with tf.GradientTape() as tape:

                # get the latent vector and its parameters from the encoder, and the reconstructions from the decoder
                mu, sigma, z, xhat = self.model(x)

                # calculate the encoder loss; this is the Kullback-Leibler distance between the true distribution of the latent
                # vector (which is assumed to be the standard normal distribution) and the distribution predicted by the encoder
                encoder_loss = - 0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf.math.log(sigma ** 2) - mu ** 2 - sigma ** 2, axis=-1))

                # calculate the decoder loss; this is the Mean Squared Error between the true values of the time series and the
                # values predicted (or reconstructed) by the decoder
                decoder_loss = tf.reduce_mean(tf.reduce_sum((tf.cast(x, tf.float32) - tf.cast(xhat, tf.float32)) ** 2, axis=-1))

                # calculate the total loss
                loss = encoder_loss + decoder_loss

                # calculate the gradient
                gradient = tape.gradient(loss, self.model.trainable_variables)

                # update the weights
                optimizer.apply_gradients(zip(gradient, self.model.trainable_variables))

                return loss

        # generate the training batches
        dataset = tf.data.Dataset.from_tensor_slices(self.x)
        dataset = dataset.cache().shuffle(self.samples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

        # train the model
        history = []
        for epoch in range(epochs):
            for x in dataset:
                loss = train_step(x)
                history.append({'epoch': 1 + epoch, 'loss': loss})
            if verbose:
                print('Epoch: {:,.0f}, Loss: {:,.4f}.'.format(1 + epoch, loss))

        return pd.DataFrame(history)

    def predict(self, data):

        tf.random.set_seed(42)

        # difference the time series
        x = data.diff().iloc[1:, :]

        # scale the time series
        x = self.scaler.transform(x)

        # get the latent vectors
        z = self.model(x)[-2].numpy()

        # organize the latent vectors in a data frame
        z = pd.concat([pd.DataFrame(np.zeros((1, 2))), pd.DataFrame(z)], axis=0, ignore_index=True)

        # get the reconstructions
        xhat = self.model(x)[-1].numpy()

        # transform the reconstructions back to the original scale and organize them in a data frame
        xhat = pd.concat([data.iloc[:1, :], pd.DataFrame(self.scaler.inverse_transform(xhat), columns=data.columns)], axis=0, ignore_index=True).cumsum(axis=0)

        return z, xhat

# HPO for VAE

In [ ]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
class VAEOptimizer:
    def __init__(self,
                 data,
                 n_trials):
        self.data = data
        self.n_trials = n_trials

    def optimize(self):
        def objective(trial):

                # Sample a given hyperparameter combination
#                dimensions = trial.suggest_int('dimensions', low=2, high=2)
                units = trial.suggest_categorical('generator_units', [(4, 16, 32, 16, 4), (32, 16, 8, 4, 2, 4, 8, 16), (20, 10, 8, 6, 4, 2), (10, 9, 8, 7, 6, 8, 6, 5, 4, 3)])
                activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
                dropout = trial.suggest_float('dropout', low=0, high=0.5, step=0.1)
                batch_size = trial.suggest_categorical('batch_size', [4000, 6000, 7500])
                learning_rate = trial.suggest_float('learning_rate', low=0.00005, high=0.001, step=0.00005)
                epochs = trial.suggest_int('epochs', low=2000, high=10000, step=1000)

                # Create the VAE model with the sampled hyperparameters
                vae_model = VAE(
                    data=self.data,
                    units=units,
                    activation=activation,
                    dropout=dropout
                )

                # Train the model and obtain training loss history
                training_history = vae_model.fit(
                    batch_size=batch_size,
                    learning_rate=learning_rate,
                    epochs=epochs,
                    verbose=False
                )
                final_loss = training_history['loss'].iloc[-1]

                # Return the final loss for optimization
                return final_loss

        # Maximize the objective function, i.e. minimize the loss
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=42))
        study.optimize(func=objective, n_trials=self.n_trials, n_jobs=-1)

        # Extract the best hyperparameters and the best loss
        self.best_params = study.best_params
        self.best_loss = study.best_value

        # Print the best hyperparameters and the best loss
        print(f'Best hyperparameters: {self.best_params}\n')
        print(f'Best loss: {self.best_loss}\n')

In [ ]:
optimizer = VAEOptimizer(
    data = x_train,
    n_trials = 10
)

In [ ]:
optimizer.optimize()

# VAE Model Training

In [ ]:
# Build the model
model = VAE(
    data=x_train,
    units=[4, 16, 32, 16, 4],
    activation='tanh',
    dropout=0.1
)

In [ ]:
# Train the model
history_vae = model.fit(
    learning_rate = 0.0009000000000000001,
    batch_size=4000,
    epochs=5000,
    verbose=True,
)

In [ ]:
# Plot Learning History Loss
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(history_regression.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.savefig('Reg_output.jpg', format='jpeg', dpi=300)
plt.show()

# Training Data Performance for VAE (Reconstruction)

In [ ]:
# Get the latent representations and the reconstructions.
L_train, xhat_train = model.predict(data = x_train)

In [ ]:
# Latent Representations
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
for i in range(L_train.shape[1]):
    plt.plot(L_train.iloc[:, i], label=f'L_train{1 + i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Latent Vectors')
plt.legend(loc='best')
plt.savefig('VAE_LV_train_output.jpg', format='jpeg', dpi=300)
plt.show()

In [ ]:
# Reconstructions
rc('font', family='Times New Roman')
num_columns = 2
num_rows = len(x.columns) // num_columns + (len(x.columns) % num_columns > 0)
fig, axes = plt.subplots(num_rows, num_columns, figsize=(8, 2 * num_rows), dpi=300)
for i, s in enumerate(x.columns):
    row = i // num_columns
    col = i % num_columns
    ax = axes[row, col] if num_rows > 1 else axes[col]
    ax.plot(x_train[s], label='actual')
    ax.plot(xhat_train[s], label='reconstructed')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(f'P {s+1}_train')
    ax.legend(loc='best')
plt.tight_layout()
plt.savefig('VAE_x_train_output.jpg', format='jpeg', dpi=300)
plt.show()

# Testing Data Performance for VAE (Reconstruction)

In [ ]:
L_test, xhat_test = model.predict(x_test)

In [ ]:
# Latent Representations
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
for i in range(L_test.shape[1]):
    plt.plot(L_test.iloc[:, i], label=f'L_test{1 + i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Latent Vectors')
plt.legend(loc='best')
plt.savefig('VAE_LV_test_output.jpg', format='jpeg', dpi=300)
plt.show()

In [ ]:
# Reconstructions
rc('font', family='Times New Roman')
num_columns = 2
num_rows = len(x.columns) // num_columns + (len(x.columns) % num_columns > 0)  # 向上取整
fig, axes = plt.subplots(num_rows, num_columns, figsize=(8, 2 * num_rows), dpi=300)
for i, s in enumerate(x.columns):
    row = i // num_columns
    col = i % num_columns
    ax = axes[row, col] if num_rows > 1 else axes[col]  # 获取当前子图对象
    ax.plot(x_test[s], label='actual')
    ax.plot(xhat_test[s], label='reconstructed')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(f'P {s+1}_test')
    ax.legend(loc='best')
plt.tight_layout()
plt.savefig('VAE_x_test_output.jpg', format='jpeg', dpi=300)
plt.show()

# Linear Regression (Training)

In [ ]:
dataReg = pd.concat([L_train, u_train, P5_train], axis=1)
dataReg = dataReg.iloc[1:]
dataReg = dataReg.reset_index(drop=True)
dataReg.columns = ['L1_train', 'L2_train', 'u1_train', 'u2_train', 'P5_train']
X_train = dataReg[['L1_train', 'L2_train', 'u1_train', 'u2_train']]
y_train = dataReg['P5_train']  # (Use P5 in next row)
X_train = X_train.iloc[:-1]
y_train = y_train.iloc[:-1]
X_train = X_train.values
y_train = y_train.values

# linear activation
def linear(x):
    return x

# Build Regression NN
regression_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=4, activation=linear),  # input layer
    tf.keras.layers.Dense(1, activation=linear)  # output layer
])

# build model
regression_model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history_regression = regression_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=False)

In [ ]:
# Plot Learning History Loss
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(history_regression.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.savefig('VAE_Reg_output.jpg', format='jpeg', dpi=300)
plt.show()

# Training Data Performance for Reg (Prediction)

In [ ]:
y_pred = regression_model.predict(X_train)

rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(y_train, label='True Values')
plt.plot(np.concatenate([[np.nan], y_pred.flatten()]), label='Predicted Values')
plt.xlabel('Sample')
plt.ylabel('P5')
plt.legend()
plt.savefig('VAE_P5_pred_train_output.jpg', format='jpeg', dpi=300)
plt.show()

# Testing Data Performance for Reg (Prediction)


In [ ]:
y_pred = regression_model.predict(X_test)

rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(y_test, label='True Values')
plt.plot(np.concatenate([[np.nan], y_pred.flatten()]), label='Predicted Values')
plt.xlabel('Sample')
plt.ylabel('P5')
plt.legend()
plt.savefig('VAE_P5_pred_test_output.jpg', format='jpeg', dpi=300)
plt.show()

# VAER Model Build

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def encoder(x, units, activation, dropout):

    # forward pass the time series through the encoder network
    for i in range(len(units)):
        h = tf.keras.layers.Dense(units=units[i], activation=activation)(x if i == 0 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # derive the mean of the latent vector
    mu = tf.keras.layers.Dense(units=2)(h)

    # derive the standard deviation of the latent vector
    sigma = tf.exp(0.5 * tf.keras.layers.Dense(units=2)(h))

    # generate the latent vector
    z = mu + sigma * tf.random.normal(shape=(tf.shape(h)[0], 2))

    return mu, sigma, z


def decoder(z, features, units, activation, dropout):

    # forward pass the latent vector through the decoder network (note that the decoder has
    # the same layers as the encoder, but in reversed order)
    for i in range(1, 1 + len(units)):
        h = tf.keras.layers.Dense(units=units[-i], activation=activation)(z if i == 1 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # forward pass the output of the decoder network through a dense layer with number of
    # units equal to the number of time series; this ensures that the reconstructions have
    # the same shape as the inputs
    xhat = tf.keras.layers.Dense(units=features)(h)

    return xhat


def discriminator(z, units, activation, dropout):

    # forward pass the latent vector through the discriminator network
    for i in range(len(units)):
        h = tf.keras.layers.Dense(units=units[i], activation=activation)(z if i == 0 else h)
        h = tf.keras.layers.Dropout(rate=dropout)(h)

    # forward pass the output of the discriminator network through a dense layer with
    # one unit and with sigmoid activation in order to get a vector of probabilities
    p = tf.keras.layers.Dense(units=1, activation='sigmoid')(h)

    return p


class VAE():
    def __init__(self, data, units, activation, dropout, u, p5):

        # difference the time series
        x = data.diff().iloc[1:, :]
        u = u.diff().iloc[1:, :]
        p5 = p5.diff().iloc[1:]

        # scale the time series
        scaler = StandardScaler()
        scaler.fit(x)
        x = scaler.transform(x)

        scaler2 = StandardScaler()
        scaler2.fit(u)
        u = scaler2.transform(u)

        p5_2d = p5.values.reshape(-1, 1)
        scaler3 = StandardScaler()
        scaler3.fit(p5_2d)
        p5 = scaler3.transform(p5_2d)

        # extract the number of samples
        samples = x.shape[0]

        # extract the number of time series
        features = x.shape[1]

        # build the model
        def build_model():

            tf.random.set_seed(42)

            # define the input
            x = tf.keras.layers.Input(shape=(features,))

            # forward pass the input through the encoder and get the latent representation
            mu, sigma, z = encoder(x, units, activation, dropout)



            # forward pass the latent representation through the decoder and get the reconstructions
            xhat = decoder(z, features, units, activation, dropout)

            # create the model
            model = tf.keras.models.Model(x, [mu, sigma, z, xhat])
            model.build(input_shape=(features,))

            return model

        # save the inputs
        self.x = x
        self.model = build_model()
        self.scaler = scaler
        self.scaler2 = scaler2
        self.scaler3 = scaler3
        self.samples = samples
        self.features = features
        self.u = u
        self.p5 = p5


    def fit(self, batch_size, learning_rate, epochs, verbose):

        tf.random.set_seed(42)

        # instantiate the optimizer
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        # define the training loop
        @tf.function
        def train_step(x):
            with tf.GradientTape() as tape:

                # get the latent vector and its parameters from the encoder, and the reconstructions from the decoder
                mu, sigma, z, xhat = self.model(x)

                # calculate the encoder loss; this is the Kullback-Leibler distance between the true distribution of the latent
                # vector (which is assumed to be the standard normal distribution) and the distribution predicted by the encoder
                encoder_loss = - 0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf.math.log(sigma ** 2) - mu ** 2 - sigma ** 2, axis=-1))

                # calculate the decoder loss; this is the Mean Squared Error between the true values of the time series and the
                # values predicted (or reconstructed) by the decoder
                decoder_loss = tf.reduce_mean(tf.reduce_sum((tf.cast(x, tf.float32) - tf.cast(xhat, tf.float32)) ** 2, axis=-1))

                # calculate the total loss
                loss = encoder_loss + decoder_loss

                # calculate the gradient
                gradient = tape.gradient(loss, self.model.trainable_variables)

                # update the weights
                optimizer.apply_gradients(zip(gradient, self.model.trainable_variables))

                return loss

        # generate the training batches
        dataset = tf.data.Dataset.from_tensor_slices(self.x)
        dataset = dataset.cache().shuffle(self.samples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

        # train the model
        history = []
        for epoch in range(epochs):
            for x in dataset:
                loss = train_step(x)
                history.append({'epoch': 1 + epoch, 'loss': loss})
            if verbose:
                print('Epoch: {:,.0f}, Loss: {:,.4f}.'.format(1 + epoch, loss))

        return pd.DataFrame(history)

    def predict(self, data):

        tf.random.set_seed(42)

        # difference the time series
        x = data.diff().iloc[1:, :]

        # scale the time series
        x = self.scaler.transform(x)

        # get the latent vectors
        z = self.model(x)[-2].numpy()

        # organize the latent vectors in a data frame
        z = pd.concat([pd.DataFrame(np.zeros((1, 2))), pd.DataFrame(z)], axis=0, ignore_index=True)

        # get the reconstructions
        xhat = self.model(x)[-1].numpy()

        # transform the reconstructions back to the original scale and organize them in a data frame
        xhat = pd.concat([data.iloc[:1, :], pd.DataFrame(self.scaler.inverse_transform(xhat), columns=data.columns)], axis=0, ignore_index=True).cumsum(axis=0)

        return z, xhat

In [ ]:
class VAEWithRegression(VAE):
    def __init__(self, data, units, activation, dropout, u, p5, regression_hidden_units):
        super().__init__(data, units, activation, dropout, u, p5)
        self.regression_model = None
        self.regression_hidden_units = regression_hidden_units
        self.build_regression_model()

    def build_regression_model(self):
            z_input = tf.keras.layers.Input(shape=(2,))
            u_input = tf.keras.layers.Input(shape=(2,))
            regression_input = tf.keras.layers.Concatenate()([z_input, u_input])

            h = regression_input
            for units in self.regression_hidden_units:
                h = tf.keras.layers.Dense(units, activation='linear')(h)
            p5_pred = tf.keras.layers.Dense(1, activation='linear')(h)


            self.regression_model = tf.keras.models.Model([z_input, u_input], p5_pred)


    def fit_combined(self, batch_size, learning_rate, epochs, verbose):
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        @tf.function
        def train_step_combined(x, u, p5):
            with tf.GradientTape() as tape:
                mu, sigma, z, xhat = self.model(x)
                p5_pred = self.regression_model([z, u])

                encoder_loss = - 0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf.math.log(sigma ** 2) - mu ** 2 - sigma ** 2, axis=-1))
                decoder_loss = tf.reduce_mean(tf.reduce_sum((tf.cast(x, tf.float32) - tf.cast(xhat, tf.float32)) ** 2, axis=-1))
                vae_loss = encoder_loss + decoder_loss
                regression_loss = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.mean_squared_error(p5, p5_pred), axis=-1))
                total_loss = vae_loss + regression_loss

                gradients = tape.gradient(total_loss, self.model.trainable_variables + self.regression_model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, self.model.trainable_variables + self.regression_model.trainable_variables))

                return total_loss

        dataset = tf.data.Dataset.from_tensor_slices((self.x, self.u, self.p5))
        dataset = dataset.cache().shuffle(self.samples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

        history = []
        for epoch in range(epochs):
            for x, u, p5 in dataset:
                loss = train_step_combined(x, u, p5)
                history.append({'epoch': epoch + 1, 'loss': loss.numpy()})
            if verbose:
                print('Epoch: {:,.0f}, Loss: {:,.4f}.'.format(epoch + 1, loss))

        return pd.DataFrame(history)

    def predict_combined(self, data, u_data):
        z, xhat = self.predict(data)
        p5_pred = self.regression_model([z.values, u_data.values]).numpy()
        p5_pred = pd.concat([pd.DataFrame(np.zeros((1, 1))), pd.DataFrame(self.scaler3.inverse_transform(p5_pred))], axis=0, ignore_index=True)

        return z, xhat, p5_pred

# HPO for VAER

In [ ]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
class VAEROptimizer:
    def __init__(self,
                 data,
                 u,
                 p5,
                 regression_hidden_units,
                 n_trials):
        self.data = data
        self.u = u
        self.p5 = p5
        self.n_trials = n_trials
        self.regression_hidden_units = regression_hidden_units

    def optimize(self):
        def objective(trial):

                # Sample a given hyperparameter combination
#                dimensions = trial.suggest_int('dimensions', low=2, high=2)
                units = trial.suggest_categorical('generator_units', [(4, 16, 32, 16, 4), (32, 16, 8, 4, 2, 4, 8, 16), (20, 10, 8, 6, 4, 2), (10, 9, 8, 7, 6, 8, 6, 5, 4, 3)])
                activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
                dropout = trial.suggest_float('dropout', low=0, high=0.5, step=0.1)
                batch_size = trial.suggest_categorical('batch_size', [5500, 6500, 7500])
                learning_rate = trial.suggest_float('learning_rate', low=0.00005, high=0.001, step=0.00005)
                epochs = trial.suggest_int('epochs', low=2000, high=10000, step=1000)

                # Create the VAER model with the sampled hyperparameters
                vaer_model = VAEWithRegression(
                    data=self.data,
                    units=units,
                    activation=activation,
                    dropout=dropout,
                    u=self.u,
                    p5=self.p5,
                    regression_hidden_units = self.regression_hidden_units # Adjust as needed
                )

                # Train the model and obtain training loss history
                training_history = vaer_model.fit_combined(
                    batch_size=batch_size,
                    learning_rate=learning_rate,
                    epochs=epochs,
                    verbose=False
                )
                final_loss = training_history['loss'].iloc[-1]

                # Return the final loss for optimization
                return final_loss

        # Maximize the objective function, i.e. minimize the loss
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=42))
        study.optimize(func=objective, n_trials=self.n_trials, n_jobs=-1)

        # Extract the best hyperparameters and the best loss
        self.best_params = study.best_params
        self.best_loss = study.best_value

        # Print the best hyperparameters and the best loss
        print(f'Best hyperparameters: {self.best_params}\n')
        print(f'Best loss: {self.best_loss}\n')

In [ ]:
optimizer = VAEROptimizer(
    data = x_train,
    u=u_train,
    p5=P5_train,
    regression_hidden_units=[10, 5],
    n_trials = 10
)

In [ ]:
optimizer.optimize()

# VAER Model Training

In [ ]:
# Build the model
combined_model = VAEWithRegression(
    data=x_train,
    units=[4, 16, 32, 16, 4],
    activation='tanh',
    dropout=0.4,
    u=u_train,
    p5=P5_train,
    regression_hidden_units=[10, 5]  # Adjust as needed
)

In [ ]:
# Training
history = combined_model.fit_combined(
    batch_size=7500,
    learning_rate=0.001,
    epochs=8000,
    verbose=True
)

In [ ]:
# Plot Learning History Loss
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(history['epoch'], history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Total Loss')
plt.savefig('VAER_loss_output.jpg', format='jpeg', dpi=300)
plt.show()

# Training Data Performance for VAER (Reconstruction)

In [ ]:
L_train, xhat_train, p5_pred_train = combined_model.predict_combined(data = x_train, u_data = u_train)

In [ ]:
# Latent Representations
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
for i in range(L_train.shape[1]):
    plt.plot(L_train.iloc[:, i], label=f'L_train{1 + i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Latent Vectors')
plt.legend(loc='best')
plt.savefig('VAER_LV_train_output.jpg', format='jpeg', dpi=300)
plt.show()

In [ ]:
# Reconstructions
rc('font', family='Times New Roman')
num_columns = 2
num_rows = len(x.columns) // num_columns + (len(x.columns) % num_columns > 0)
fig, axes = plt.subplots(num_rows, num_columns, figsize=(8, 2 * num_rows), dpi=300)
for i, s in enumerate(x.columns):
    row = i // num_columns
    col = i % num_columns
    ax = axes[row, col] if num_rows > 1 else axes[col]
    ax.plot(x_train[s], label='actual')
    ax.plot(xhat_train[s], label='reconstructed')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(f'P {s+1}_train')
    ax.legend(loc='best')
plt.tight_layout()
plt.savefig('VAER_x_train_output.jpg', format='jpeg', dpi=300)
plt.show()

# Testing Data Performance for VAER (Reconstruction)

In [ ]:
L_test, xhat_test, p5_pred_test = combined_model.predict_combined(x_test, u_test)

In [ ]:
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
for i in range(L_test.shape[1]):
    plt.plot(L_test.iloc[:, i], label=f'L_test{1 + i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Latent Vectors')
plt.legend(loc='best')
plt.savefig('VAER_LV_test_output.jpg', format='jpeg', dpi=300)
plt.show()

In [ ]:
rc('font', family='Times New Roman')
num_columns = 2
num_rows = len(x.columns) // num_columns + (len(x.columns) % num_columns > 0)  # 向上取整
fig, axes = plt.subplots(num_rows, num_columns, figsize=(8, 2 * num_rows), dpi=300)
for i, s in enumerate(x.columns):
    row = i // num_columns
    col = i % num_columns
    ax = axes[row, col] if num_rows > 1 else axes[col]  # 获取当前子图对象

    ax.plot(x_test[s], label='actual')
    ax.plot(xhat_test[s], label='reconstructed')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(f'P {s+1}_test')
    ax.legend(loc='best')
plt.tight_layout()
plt.savefig('VAER_x_test_output.jpg', format='jpeg', dpi=300)
plt.show()

# Training Data Performance for VAER (Prediction)

In [ ]:
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(P5_train, label='True Values')
plt.plot(p5_pred_train, label='Predicted Values')
plt.xlabel('Sample')
plt.ylabel('P5')
plt.legend()
plt.savefig('VAER_P5_pred_train_output.jpg', format='jpeg', dpi=300)
plt.show()

# Testing Data Performance for VAER (Prediction)

In [ ]:
plt.show()
rc('font', family='Times New Roman')
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(P5_test, label='True Values')
plt.plot(p5_pred_test, label='Predicted Values')
plt.xlabel('Sample')
plt.ylabel('P5')
plt.legend()
plt.savefig('VAER_P5_pred_test_output.jpg', format='jpeg', dpi=300)
plt.show()